In [ ]:
import pandas as pd
import numpy as np

class EncoderReverser:
    """
    معکوس‌سازی عملیات Encoding روی یک ستون.

    Parameters
    ----------
    method : str
        نام متد encoding که باید معکوس شود.
        پشتیبانی شده: 'OneHotEncoder', 'LabelEncoder', 'BinaryEncoder',
        'OrdinalEncoder', 'LabelBinarizer', 'CyclicalEncoder'
        (FrequencyEncoder, TargetEncoder, WoE غیرقابل معکوس هستند)
    config : dict
        تنظیمات شامل پارامترهای fitted شده (مانند categories_, classes_, ...)
    """
    def __init__(self, method, config):
        self.method = method
        self.config = config
        self.encoder_ = None

    def transform(self, dataset, column, encoded_columns=None):
        """
        اعمال معکوس‌سازی روی ستون مشخص و افزودن نتیجه به دیتاست.

        Parameters
        ----------
        dataset : pd.DataFrame
            دیتاست ورودی.
        column : str
            نام ستون اصلی (قبل از encoding).
        encoded_columns : list, optional
            برای روش‌هایی که چند ستون خروجی دارند (مثل one-hot)، لیست نام ستون‌های کدگذاری شده.
            اگر None باشد، از روی نام ستون‌ها حدس زده می‌شود.

        Returns
        -------
        pd.DataFrame
            دیتاست با ستون جدید '{column}_decoded'.
        """
        if not isinstance(dataset, pd.DataFrame):
            raise TypeError("dataset must be a pandas DataFrame")

        dataset = dataset.copy()

        if self.method == 'OneHotEncoder':
            return self._reverse_onehot(dataset, column, encoded_columns)
        elif self.method == 'LabelEncoder':
            return self._reverse_label(dataset, column)
        elif self.method == 'BinaryEncoder':
            return self._reverse_binary(dataset, column, encoded_columns)
        elif self.method == 'OrdinalEncoder':
            return self._reverse_ordinal(dataset, column)
        elif self.method == 'LabelBinarizer':
            return self._reverse_label_binarizer(dataset, column, encoded_columns)
        elif self.method == 'CyclicalEncoder':
            return self._reverse_cyclical(dataset, column, encoded_columns)
        elif self.method in ['FrequencyEncoder', 'TargetEncoder', 'Weight of Evidence (WoE)']:
            raise NotImplementedError(f"{self.method} is not reversible (multiple categories can map to the same value).")
        else:
            raise ValueError(f"Unknown encoding method: {self.method}")

    def _reverse_onehot(self, dataset, column, encoded_columns):
        """معکوس OneHotEncoder: فرض می‌کنیم ستون‌های one-hot به صورت {column}_cat_{value} نامیده شده‌اند."""
        categories = self.config.get('categories')
        if categories is None:
            raise ValueError("OneHotEncoder requires 'categories' in config (list of category names).")

        # اگر encoded_columns داده نشده، از روی الگو پیدا کن
        if encoded_columns is None:
            encoded_columns = [col for col in dataset.columns if col.startswith(f"{column}_cat_")]

        if not encoded_columns:
            raise ValueError(f"No encoded columns found for '{column}'")

        # ایجاد ماتریس one-hot (فرض می‌کنیم مقادیر 0/1 هستند)
        one_hot = dataset[encoded_columns].values
        # پیدا کردن اندیس ماکزیمم (اگر چند تا 1 باشد، اولی را می‌گیرد)
        indices = np.argmax(one_hot, axis=1)
        # تبدیل به دسته
        decoded = [categories[i] for i in indices]
        dataset[f"{column}_decoded"] = decoded
        return dataset

    def _reverse_label(self, dataset, column):
        """معکوس LabelEncoder."""
        classes = self.config.get('classes')
        if classes is None:
            raise ValueError("LabelEncoder requires 'classes' in config (list of class names).")

        # ستون باید اعداد صحیح باشد
        col_data = dataset[column].values
        # ممکن است ستون به صورت object باشد، سعی می‌کنیم به int تبدیل کنیم
        try:
            indices = col_data.astype(int)
        except:
            raise ValueError(f"Column '{column}' must contain integer codes for LabelEncoder reversal.")

        decoded = [classes[i] if 0 <= i < len(classes) else np.nan for i in indices]
        dataset[f"{column}_decoded"] = decoded
        return dataset

    def _reverse_binary(self, dataset, column, encoded_columns):
        """معکوس BinaryEncoder: نیاز به mapping از باینری به دسته."""
        binary_map = self.config.get('binary_map')
        if binary_map is None:
            raise ValueError("BinaryEncoder requires 'binary_map' in config (dict mapping binary string to category).")

        if encoded_columns is None:
            # فرض می‌کنیم ستون‌ها به صورت {column}_bin_{i} هستند
            encoded_columns = sorted([col for col in dataset.columns if col.startswith(f"{column}_bin_")])

        if not encoded_columns:
            raise ValueError(f"No encoded columns found for '{column}'")

        # ساخت رشته باینری از ستون‌ها
        binary_data = dataset[encoded_columns].astype(int).values
        binary_strings = [''.join(map(str, row)) for row in binary_data]

        decoded = [binary_map.get(b, np.nan) for b in binary_strings]
        dataset[f"{column}_decoded"] = decoded
        return dataset

    def _reverse_ordinal(self, dataset, column):
        """معکوس OrdinalEncoder (مشابه LabelEncoder)."""
        return self._reverse_label(dataset, column)

    def _reverse_label_binarizer(self, dataset, column, encoded_columns):
        """معکوس LabelBinarizer (شبیه one-hot)."""
        classes = self.config.get('classes')
        if classes is None:
            raise ValueError("LabelBinarizer requires 'classes' in config.")

        if encoded_columns is None:
            encoded_columns = [col for col in dataset.columns if col.startswith(f"{column}_bin_")]

        if not encoded_columns:
            raise ValueError(f"No encoded columns found for '{column}'")

        one_hot = dataset[encoded_columns].values
        indices = np.argmax(one_hot, axis=1)
        decoded = [classes[i] for i in indices]
        dataset[f"{column}_decoded"] = decoded
        return dataset

    def _reverse_cyclical(self, dataset, column, encoded_columns):
        """معکوس CyclicalEncoder: از sin و cos به زاویه و سپس به مقدار اصلی."""
        # فرض می‌کنیم ستون‌ها به صورت {column}_sin و {column}_cos هستند
        if encoded_columns is None:
            sin_col = f"{column}_sin"
            cos_col = f"{column}_cos"
        else:
            sin_col, cos_col = encoded_columns[0], encoded_columns[1]

        if sin_col not in dataset.columns or cos_col not in dataset.columns:
            raise ValueError(f"Required sin/cos columns not found for '{column}'")

        sin_vals = dataset[sin_col].values
        cos_vals = dataset[cos_col].values

        # محاسبه زاویه
        angles = np.arctan2(sin_vals, cos_vals)  # نتیجه در [-π, π]
        # نرمال‌سازی به [0, 2π]
        angles = (angles + 2 * np.pi) % (2 * np.pi)

        # پارامترهای تبدیل (حداکثر مقدار و حداقل)
        max_val = self.config.get('max_value')
        min_val = self.config.get('min_value', 0)
        if max_val is None:
            raise ValueError("CyclicalEncoder requires 'max_value' in config.")

        # تبدیل زاویه به مقدار اصلی (خطی)
        decoded = (angles / (2 * np.pi)) * (max_val - min_val) + min_val
        dataset[f"{column}_decoded"] = decoded
        return dataset